In [1]:
import warnings
warnings.simplefilter('ignore')

import os
import re
import gc

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
pd.set_option('max_rows', 200)
pd.set_option('float_format', lambda x: '%.3f' % x)

from tqdm.notebook import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score

import lightgbm as lgb

In [2]:
!pip install tqdm --upgrade

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Requirement already up-to-date: tqdm in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (4.62.3)


# 读取数据

In [3]:
train_data = pd.read_csv('../raw_data/train_public.csv')

train_data['target'] = 1

print(train_data.shape)
train_data.head()

(10000, 40)


loan_id  user_id  total_loan  year_of_loan  interest  monthly_payment  \
0  1040418   240418   31818.182             3    11.466         1174.910   
1  1025197   225197   28000.000             5    16.841          670.690   
2  1009360   209360   17272.727             3     8.900          603.320   
3  1039708   239708   20000.000             3     4.788          602.300   
4  1027483   227483   15272.727             3    12.790          470.310   

  class employer_type         industry  work_year  house_exist  censor_status  \
0     C          政府机构              金融业    3 years            0              1   
1     C          政府机构              金融业  10+ years            0              2   
2     A          政府机构        公共服务、社会组织  10+ years            1              0   
3     A         世界五百强           文化和体育业    6 years            0              1   
4     C          政府机构  信息传输、软件和信息技术服务业   < 1 year            2              1   

  issue_date  use  post_code  region  debt_loan_ratio  del_in_18month  \
0  2016/10/1    2        193      13            2.430               0   
1   2013/6/1    0        491      30           11.005               1   
2   2014/1/1    4        459       8            6.409               0   
3   2015/7/1    0        157       8            9.205               0   
4   2016/7/1    0         38      21           15.578               0   

   scoring_low  scoring_high  known_outstanding_loan  known_dero  \
0      556.364       649.091                       3           0   
1      715.000       893.750                       3           0   
2      774.545       903.636                       5           0   
3      750.000       875.000                       3           0   
4      609.091       710.606                      15           0   

   pub_dero_bankrup  recircle_b  recircle_u  initial_list_status  app_type  \
0             0.000    7734.231      91.800                    0         0   
1             0.000   31329.000      54.800                    1         0   
2             0.000   18514.000      57.692                    1         0   
3             0.000   20707.000      42.600                    0         0   
4             0.000   14016.154      30.462                    0         0   

  earlies_credit_mon  title  policy_code     f0    f1     f2     f3     f4  \
0              1-Dec      5            1  1.000 0.000  4.000  5.000  4.000   
1             Apr-90  40642            1  7.000 0.000  4.000 45.000 22.000   
2             Oct-91    154            1  6.000 0.000  6.000 28.000 19.000   
3              1-Jun      0            1  5.000 0.000 10.000 15.000  9.000   
4              2-May      0            1 10.000 0.000  6.000 15.000  4.000   

   early_return  early_return_amount  early_return_amount_3mon  isDefault  \
0             3                 9927                     0.000          0   
1             0                    0                     0.000          0   
2             0                    0                     0.000          0   
3             0                    0                     0.000          0   
4             0                    0                     0.000          0   

   target  
0       1  
1       1  
2       1  
3       1  
4       1

In [4]:
train_internet = pd.read_csv('../raw_data/train_internet.csv')

train_internet['target'] = 0

print(train_internet.shape)
train_internet.head()

(750000, 43)


loan_id  user_id  total_loan  year_of_loan  interest  monthly_payment  \
0   119262        0   12000.000             5    11.530          264.100   
1   369815        1    8000.000             3    13.980          273.350   
2   787833        2   20000.000             5    17.990          507.760   
3   671675        3   10700.000             3    10.160          346.070   
4   245160        4    8000.000             3     8.240          251.580   

  class sub_class work_type employer_type         industry  work_year  \
0     B        B5        职员          普通企业              采矿业        NaN   
1     C        C3        其他          普通企业             国际组织  10+ years   
2     D        D2        工人          上市企业  信息传输、软件和信息技术服务业  10+ years   
3     B        B1        职员          普通企业       电力、热力生产供应业    2 years   
4     B        B1        其他          政府机构              金融业    5 years   

   house_exist  house_loan_status  censor_status  marriage  offsprings  \
0            0                  0              2         0           0   
1            0                  1              2         1           3   
2            0                  0              1         0           0   
3            2                  0              2         0           0   
4            1                  2              0         0           0   

   issue_date  use  post_code  region  debt_loan_ratio  del_in_18month  \
0  2015-06-01    0    814.000       4            5.070           1.000   
1  2010-10-01    2    240.000      21           15.040           0.000   
2  2016-08-01    0    164.000      20           17.380           1.000   
3  2013-05-01    4     48.000      10           27.870           0.000   
4  2017-04-01    4    122.000       9            3.470           0.000   

   scoring_low  scoring_high  pub_dero_bankrup  early_return  \
0      670.000       674.000             1.000             0   
1      725.000       729.000             0.000             0   
2      675.000       679.000             0.000             0   
3      710.000       714.000             0.000             0   
4      660.000       664.000             0.000             0   

   early_return_amount  early_return_amount_3mon  recircle_b  recircle_u  \
0                    0                     0.000    3855.000      23.100   
1                    0                     0.000  118632.000      99.900   
2                    0                     0.000   15670.000      72.500   
3                    0                     0.000   18859.000      78.600   
4                    0                     0.000    8337.000      67.800   

   initial_list_status earlies_credit_mon     title  policy_code    f0    f1  \
0                    0           Mar-1984     0.000        1.000 1.000 0.000   
1                    1           Jan-1992    94.000        1.000   nan   nan   
2                    0           Oct-1996     0.000        1.000 6.000 0.000   
3                    0           Jul-2000 41646.000        1.000 3.000 0.000   
4                    1           Mar-2000     4.000        1.000 3.000 0.000   

      f2     f3    f4    f5  is_default  target  
0  8.000 17.000 8.000 1.000           1       0  
1    nan    nan   nan   nan           0       0  
2 10.000  8.000 3.000 0.000           0       0  
3  4.000 11.000 6.000 0.000           0       0  
4  8.000  6.000 4.000 1.000           0       0

In [5]:
train_data['isDefault'].value_counts(dropna=True)

0    8317
1    1683
Name: isDefault, dtype: int64

In [6]:
train_internet = train_internet.rename(columns={'is_default': 'isDefault'})
train_internet['isDefault'].value_counts(dropna=True)

0    600327
1    149673
Name: isDefault, dtype: int64

# 数据整理

In [7]:
drop1 = ['sub_class', 'work_type', 'house_loan_status', 'marriage', 'offsprings', 'f5']
drop2 = ['known_outstanding_loan', 'known_dero', 'app_type']

train_internet.drop(drop1 + ['user_id'], axis=1, inplace=True)
train_data.drop(drop2 + ['user_id'], axis=1, inplace=True)

train_data = pd.concat([train_data, train_internet]).reset_index(drop=True)
print(train_data.shape)
train_data.head()

(760000, 36)


loan_id  total_loan  year_of_loan  interest  monthly_payment class  \
0  1040418   31818.182             3    11.466         1174.910     C   
1  1025197   28000.000             5    16.841          670.690     C   
2  1009360   17272.727             3     8.900          603.320     A   
3  1039708   20000.000             3     4.788          602.300     A   
4  1027483   15272.727             3    12.790          470.310     C   

  employer_type         industry  work_year  house_exist  censor_status  \
0          政府机构              金融业    3 years            0              1   
1          政府机构              金融业  10+ years            0              2   
2          政府机构        公共服务、社会组织  10+ years            1              0   
3         世界五百强           文化和体育业    6 years            0              1   
4          政府机构  信息传输、软件和信息技术服务业   < 1 year            2              1   

  issue_date  use  post_code  region  debt_loan_ratio  del_in_18month  \
0  2016/10/1    2    193.000      13            2.430           0.000   
1   2013/6/1    0    491.000      30           11.005           1.000   
2   2014/1/1    4    459.000       8            6.409           0.000   
3   2015/7/1    0    157.000       8            9.205           0.000   
4   2016/7/1    0     38.000      21           15.578           0.000   

   scoring_low  scoring_high  pub_dero_bankrup  recircle_b  recircle_u  \
0      556.364       649.091             0.000    7734.231      91.800   
1      715.000       893.750             0.000   31329.000      54.800   
2      774.545       903.636             0.000   18514.000      57.692   
3      750.000       875.000             0.000   20707.000      42.600   
4      609.091       710.606             0.000   14016.154      30.462   

   initial_list_status earlies_credit_mon     title  policy_code     f0    f1  \
0                    0              1-Dec     5.000        1.000  1.000 0.000   
1                    1             Apr-90 40642.000        1.000  7.000 0.000   
2                    1             Oct-91   154.000        1.000  6.000 0.000   
3                    0              1-Jun     0.000        1.000  5.000 0.000   
4                    0              2-May     0.000        1.000 10.000 0.000   

      f2     f3     f4  early_return  early_return_amount  \
0  4.000  5.000  4.000             3                 9927   
1  4.000 45.000 22.000             0                    0   
2  6.000 28.000 19.000             0                    0   
3 10.000 15.000  9.000             0                    0   
4  6.000 15.000  4.000             0                    0   

   early_return_amount_3mon  isDefault  target  
0                     0.000          0       1  
1                     0.000          0       1  
2                     0.000          0       1  
3                     0.000          0       1  
4                     0.000          0       1

In [8]:
# data = pd.concat([train_data, test_data])
data = train_data.copy()

print(data.shape)
data.head()

(760000, 36)


loan_id  total_loan  year_of_loan  interest  monthly_payment class  \
0  1040418   31818.182             3    11.466         1174.910     C   
1  1025197   28000.000             5    16.841          670.690     C   
2  1009360   17272.727             3     8.900          603.320     A   
3  1039708   20000.000             3     4.788          602.300     A   
4  1027483   15272.727             3    12.790          470.310     C   

  employer_type         industry  work_year  house_exist  censor_status  \
0          政府机构              金融业    3 years            0              1   
1          政府机构              金融业  10+ years            0              2   
2          政府机构        公共服务、社会组织  10+ years            1              0   
3         世界五百强           文化和体育业    6 years            0              1   
4          政府机构  信息传输、软件和信息技术服务业   < 1 year            2              1   

  issue_date  use  post_code  region  debt_loan_ratio  del_in_18month  \
0  2016/10/1    2    193.000      13            2.430           0.000   
1   2013/6/1    0    491.000      30           11.005           1.000   
2   2014/1/1    4    459.000       8            6.409           0.000   
3   2015/7/1    0    157.000       8            9.205           0.000   
4   2016/7/1    0     38.000      21           15.578           0.000   

   scoring_low  scoring_high  pub_dero_bankrup  recircle_b  recircle_u  \
0      556.364       649.091             0.000    7734.231      91.800   
1      715.000       893.750             0.000   31329.000      54.800   
2      774.545       903.636             0.000   18514.000      57.692   
3      750.000       875.000             0.000   20707.000      42.600   
4      609.091       710.606             0.000   14016.154      30.462   

   initial_list_status earlies_credit_mon     title  policy_code     f0    f1  \
0                    0              1-Dec     5.000        1.000  1.000 0.000   
1                    1             Apr-90 40642.000        1.000  7.000 0.000   
2                    1             Oct-91   154.000        1.000  6.000 0.000   
3                    0              1-Jun     0.000        1.000  5.000 0.000   
4                    0              2-May     0.000        1.000 10.000 0.000   

      f2     f3     f4  early_return  early_return_amount  \
0  4.000  5.000  4.000             3                 9927   
1  4.000 45.000 22.000             0                    0   
2  6.000 28.000 19.000             0                    0   
3 10.000 15.000  9.000             0                    0   
4  6.000 15.000  4.000             0                    0   

   early_return_amount_3mon  isDefault  target  
0                     0.000          0       1  
1                     0.000          0       1  
2                     0.000          0       1  
3                     0.000          0       1  
4                     0.000          0       1

In [9]:
data['issue_date'] = pd.to_datetime(data['issue_date'])
data['issue_mon'] = data['issue_date'].dt.year * 100 + data['issue_date'].dt.month
data.drop(['issue_date'], axis=1, inplace=True)

In [10]:
data['class'] = data['class'].map({
    'A': 0, 'B': 1, 'C': 2, 'D': 3,
    'E': 4, 'F': 5, 'G': 6
})

In [11]:
lbe = LabelEncoder()
data['employer_type'] = lbe.fit_transform(data['employer_type'])

In [12]:
lbe = LabelEncoder()
data['industry'] = lbe.fit_transform(data['industry'])

In [13]:
data['work_year'] = data['work_year'].map({
    '< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3, '4 years': 4,
    '5 years': 5, '6 years': 6, '7 years': 7, '8 years': 8, '9 years': 9,
    '10+ years': 10
})

data['work_year'].fillna(-1, inplace=True)

In [14]:
def clean_mon(x):
    mons = {'jan':1, 'feb':2, 'mar':3, 'apr':4,  'may':5,  'jun':6,
            'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12}
    year_group = re.search('(\d+)', x)
    if year_group:
        year = int(year_group.group(1))
        if year < 22:
            year += 2000
        elif 100 > year > 22:
            year += 1900
        else:
            pass
    else:
        year = 2022
        
    month_group = re.search('([a-zA-Z]+)', x)
    if month_group:
        mon = month_group.group(1).lower()
        month = mons[mon]
    else:
        month = 0
        
    return year*100 + month

data['earlies_credit_mon'] = data['earlies_credit_mon'].apply(lambda x: clean_mon(x))

In [15]:
data.head()

loan_id  total_loan  year_of_loan  interest  monthly_payment  class  \
0  1040418   31818.182             3    11.466         1174.910      2   
1  1025197   28000.000             5    16.841          670.690      2   
2  1009360   17272.727             3     8.900          603.320      0   
3  1039708   20000.000             3     4.788          602.300      0   
4  1027483   15272.727             3    12.790          470.310      2   

   employer_type  industry  work_year  house_exist  censor_status  use  \
0              3        13      3.000            0              1    2   
1              3        13     10.000            0              2    0   
2              3         3     10.000            1              0    4   
3              1        10      6.000            0              1    0   
4              3         2      0.000            2              1    0   

   post_code  region  debt_loan_ratio  del_in_18month  scoring_low  \
0    193.000      13            2.430           0.000      556.364   
1    491.000      30           11.005           1.000      715.000   
2    459.000       8            6.409           0.000      774.545   
3    157.000       8            9.205           0.000      750.000   
4     38.000      21           15.578           0.000      609.091   

   scoring_high  pub_dero_bankrup  recircle_b  recircle_u  \
0       649.091             0.000    7734.231      91.800   
1       893.750             0.000   31329.000      54.800   
2       903.636             0.000   18514.000      57.692   
3       875.000             0.000   20707.000      42.600   
4       710.606             0.000   14016.154      30.462   

   initial_list_status  earlies_credit_mon     title  policy_code     f0  \
0                    0              200112     5.000        1.000  1.000   
1                    1              199004 40642.000        1.000  7.000   
2                    1              199110   154.000        1.000  6.000   
3                    0              200106     0.000        1.000  5.000   
4                    0              200205     0.000        1.000 10.000   

     f1     f2     f3     f4  early_return  early_return_amount  \
0 0.000  4.000  5.000  4.000             3                 9927   
1 0.000  4.000 45.000 22.000             0                    0   
2 0.000  6.000 28.000 19.000             0                    0   
3 0.000 10.000 15.000  9.000             0                    0   
4 0.000  6.000 15.000  4.000             0                    0   

   early_return_amount_3mon  isDefault  target  issue_mon  
0                     0.000          0       1     201610  
1                     0.000          0       1     201306  
2                     0.000          0       1     201401  
3                     0.000          0       1     201507  
4                     0.000          0       1     201607

# 模型

In [16]:
train = data.copy()
test  = data[data['target'] == 0].copy()

ycol = 'target'
feature_names = list(
    filter(lambda x: x not in [ycol, 'loan_id', 'policy_code'], train.columns))

model = lgb.LGBMClassifier(objective='binary',
                           boosting_type='gbdt',
                           tree_learner='serial',
                           num_leaves=32,
                           max_depth=6,
                           learning_rate=0.1,
                           n_estimators=10000,
                           subsample=0.8,
                           feature_fraction=0.6,
                           reg_alpha=0.5,
                           reg_lambda=0.5,
                           random_state=2021,
                           is_unbalance=True,
                           metric='auc')


oof = []
prediction = test[['loan_id']]
prediction[ycol] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2021)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train[feature_names], train[ycol])):
    X_train = train.iloc[trn_idx][feature_names]
    Y_train = train.iloc[trn_idx][ycol]

    X_val = train.iloc[val_idx][feature_names]
    Y_val = train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(X_train,
                          Y_train,
                          eval_names=['train', 'valid'],
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=500,
                          eval_metric='auc',
                          early_stopping_rounds=50)

    pred_val = lgb_model.predict_proba(
        X_val, num_iteration=lgb_model.best_iteration_)
    df_oof = train.iloc[val_idx][['loan_id', ycol]].copy()
    df_oof['pred'] = pred_val[:, 1]
    oof.append(df_oof)

    pred_test = lgb_model.predict_proba(
        test[feature_names], num_iteration=lgb_model.best_iteration_)
    prediction[ycol] += pred_test[:, 1] / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()
    


Fold_1 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[38]	train's auc: 1	valid's auc: 0.999995

Fold_2 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[36]	train's auc: 0.999999	valid's auc: 1

Fold_3 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[39]	train's auc: 0.999999	valid's auc: 1

Fold_4 Training ================================

[LightGBM]

In [17]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

column  importance
0                scoring_low     324.600
1               scoring_high     165.200
2                   interest     154.800
3        early_return_amount      95.600
4                 recircle_u      83.600
5                 total_loan      70.000
6   early_return_amount_3mon      64.800
7                      class      48.600
8               early_return      47.400
9                         f0      37.400
10                recircle_b      29.800
11        earlies_credit_mon      29.000
12           debt_loan_ratio      25.800
13                 issue_mon      21.400
14           monthly_payment      21.200
15            del_in_18month      19.400
16          pub_dero_bankrup      15.600
17              year_of_loan      14.600
18             employer_type      12.000
19                        f3      11.800
20                 post_code      11.400
21                        f2      10.200
22                 isDefault       9.000
23                     title       7.600
24                    region       7.000
25                        f4       6.200
26               house_exist       4.600
27                  industry       3.600
28                       use       2.800
29             censor_status       2.600
30                 work_year       2.200
31       initial_list_status       1.000
32                        f1       0.200

In [18]:
oof = pd.concat(oof)
print('roc_auc_score:', roc_auc_score(oof['target'], oof['pred']))

roc_auc_score: 0.9999992305333333


In [19]:
prediction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750000 entries, 10000 to 759999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   loan_id  750000 non-null  int64  
 1   target   750000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 17.2 MB


In [20]:
prediction['target'].describe()

count   750000.000
mean         0.011
std          0.016
min          0.001
25%          0.006
50%          0.007
75%          0.008
max          0.999
Name: target, dtype: float64

In [21]:
save_list = prediction[prediction['target'] >= 0.1]['loan_id'].tolist()
len(save_list)

4248

In [22]:
train_internet = pd.read_csv('../raw_data/train_internet.csv')
train_internet = train_internet[train_internet['loan_id'].isin(save_list)].copy()
print(train_internet.shape)
train_internet.head()

(4248, 42)


loan_id  user_id  total_loan  year_of_loan  interest  monthly_payment  \
240   414778      240   16000.000             5    19.420          418.760   
359   697588      359   20000.000             5     7.340          399.250   
403    88991      403    2400.000             3    15.610           83.920   
491   210862      491   25200.000             5    12.990          573.250   
665   365687      665   14250.000             3    13.990          486.970   

    class sub_class work_type employer_type         industry  work_year  \
240     D        D3        其他          普通企业       电力、热力生产供应业  10+ years   
359     A        A4        其他          政府机构  信息传输、软件和信息技术服务业        NaN   
403     D        D1        职员          上市企业       电力、热力生产供应业    2 years   
491     C        C2        职员          普通企业           批发和零售业    8 years   
665     C        C4        工人          普通企业  信息传输、软件和信息技术服务业    9 years   

     house_exist  house_loan_status  censor_status  marriage  offsprings  \
240            2                  2              2         0           0   
359            0                  2              0         0           0   
403            1                  1              1         1           5   
491            0                  1              0         0           0   
665            0                  0              1         1           2   

     issue_date  use  post_code  region  debt_loan_ratio  del_in_18month  \
240  2018-03-01    4    114.000      13           31.870           0.000   
359  2018-02-01    2     69.000      26           10.210           0.000   
403  2015-05-01    4    402.000       0           20.900           0.000   
491  2016-03-01    0    129.000      30           14.350           0.000   
665  2015-03-01    0    109.000      21           22.200           0.000   

     scoring_low  scoring_high  pub_dero_bankrup  early_return  \
240      720.000       724.000             0.000             0   
359      825.000       829.000             0.000             0   
403      780.000       784.000             0.000             0   
491      810.000       814.000             0.000             0   
665      720.000       724.000             0.000             0   

     early_return_amount  early_return_amount_3mon  recircle_b  recircle_u  \
240                    0                     0.000    7582.000      70.900   
359                    0                     0.000   10274.000       6.900   
403                    0                     0.000     752.000       4.400   
491                    0                     0.000   17915.000      27.800   
665                    0                     0.000   42906.000      69.100   

     initial_list_status earlies_credit_mon  title  policy_code    f0    f1  \
240                    0           Apr-1997  4.000        1.000 4.000 0.000   
359                    0           Sep-1986  5.000        1.000 4.000 0.000   
403                    1           Mar-2009  4.000        1.000 4.000 0.000   
491                    0           Mar-1989  0.000        1.000 5.000 0.000   
665                    1           Feb-1994  0.000        1.000 7.000 0.000   

       f2     f3     f4    f5  is_default  
240 8.000  7.000  3.000 0.000           1  
359 4.000 25.000 19.000 0.000           0  
403 6.000  5.000  4.000 0.000           0  
491 5.000 16.000 12.000 0.000           1  
665 0.000 14.000  6.000 0.000           0

In [23]:
train_internet = train_internet.rename(columns={'is_default': 'isDefault'})
train_internet['isDefault'].value_counts()

0    3239
1    1009
Name: isDefault, dtype: int64

In [24]:
train_internet[['total_loan']]=train_internet[['total_loan']].astype(np.int64)

In [25]:
train_internet.to_csv(f'../user_data/train_internet_{len(train_internet)}.csv', index=False)